In [6]:
!pip install transformers tensorflow tf-keras


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 38.9 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import tensorflow as tf
from transformers import AutoTokenizer, TFBertForSequenceClassification

# Load BERTimbau tokenizer and model
model_name = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Binary classification



2025-01-10 01:00:27.428918: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 91527168 exceeds 10% of free system memory.
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at neuralmind/bert-base-portuguese-cased and are newly initialized: ['classifier', 'bert/pooler/dense/bias:0', 'bert/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# Sample data
texts = ["Eu amo este filme.", "O filme foi terrível."]
labels = [1, 0]

# Tokenize inputs
encoded_inputs = tokenizer(
    texts,
    max_length=64,
    padding="max_length",
    truncation=True,
    return_tensors="tf"
)



In [9]:
# Prepare the dataset
X = {
    "input_ids": encoded_inputs["input_ids"],
    "attention_mask": encoded_inputs["attention_mask"]
}
y = tf.convert_to_tensor(labels)


def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
        pt = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
        loss = -alpha * (1 - pt) ** gamma * tf.math.log(pt)
        return tf.reduce_mean(loss)
    return focal_loss_fixed


    # Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss=focal_loss(gamma=2.0, alpha=0.25),
    metrics=['accuracy']
)
# Convert to TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X, y))
train_dataset = train_dataset.batch(2)  # Set batch size


model.fit(train_dataset, epochs=3)


# Sample test texts
test_texts = ["Este filme é ótimo!", "Eu não gostei do filme."]
test_labels = [1, 0]

# Tokenize test inputs
test_inputs = tokenizer(
    test_texts,
    max_length=64,
    padding="max_length",
    truncation=True,
    return_tensors="tf"
)

# Make predictions
predictions = model.predict({
    "input_ids": test_inputs["input_ids"],
    "attention_mask": test_inputs["attention_mask"]
})

# Convert logits to class labels
predicted_classes = tf.argmax(predictions.logits, axis=1)
print("Predictions:", predicted_classes.numpy())

Epoch 1/3


2025-01-10 00:45:25.667664: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 91527168 exceeds 10% of free system memory.


1/1 [==============================] - 24s 24s/step - loss: 1.2075 - accuracy: 1.0000
Epoch 2/3
1/1 [==============================] - 2s 2s/step - loss: 1.0779 - accuracy: 0.5000
Epoch 3/3
1/1 [==============================] - 2s 2s/step
Predictions: [1 1]


In [1]:
import tensorflow as tf
from transformers import AutoTokenizer, TFBertForSequenceClassification
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

# Load BERTimbau tokenizer and model
model_name = "neuralmind/bert-base-portuguese-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_default = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Model without Focal Loss
model_focal = TFBertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Model with Focal Loss


2025-01-10 01:01:08.466098: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-10 01:01:08.469627: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-10 01:01:08.478556: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736470868.492788   13206 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736470868.497069   13206 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-10 01:01:08.513794: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [2]:
# Sample data
texts = ["Eu amo este filme.", "O filme foi terrível.", "Este é o melhor livro que já li.", "Não gostei da comida."]
labels = [1, 0, 1, 0]

# Tokenize inputs
encoded_inputs = tokenizer(
    texts,
    max_length=64,
    padding="max_length",
    truncation=True,
    return_tensors="tf"
)

# Prepare the dataset
X = {
    "input_ids": encoded_inputs["input_ids"],
    "attention_mask": encoded_inputs["attention_mask"]
}
y = tf.convert_to_tensor(labels)


In [3]:
def focal_loss(gamma=2.0, alpha=0.25):
    def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
        pt = tf.where(tf.equal(y_true, 1), y_pred, 1 - y_pred)
        return -alpha * (1 - pt) ** gamma * tf.math.log(pt)
    return loss


In [4]:
# Compile the default model (without Focal Loss)
model_default.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Compile the model with Focal Loss
model_focal.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss=focal_loss(gamma=2.0, alpha=0.25),
    metrics=['accuracy']
)


In [5]:
model_focal.fit(X, y, epochs=3, batch_size=2)


Epoch 1/3


2/2 [==============================] - 23s 2s/step - loss: 0.4516 - accuracy: 0.0000e+00
Epoch 2/3
2/2 [==============================] - 3s 2s/step - loss: 0.2072 - accuracy: 0.5000
Epoch 3/3
2/2 [==============================] - 3s 2s/step - loss: 0.1171 - accuracy: 0.5000


In [6]:
test_texts = ["Gostei muito do filme.", "O atendimento foi péssimo."]
test_labels = [1, 0]

# Tokenize test data
test_inputs = tokenizer(
    test_texts,
    max_length=64,
    padding="max_length",
    truncation=True,
    return_tensors="tf"
)

# Convert to TensorFlow Dataset
test_X = {
    "input_ids": test_inputs["input_ids"],
    "attention_mask": test_inputs["attention_mask"]
}
test_y = tf.convert_to_tensor(test_labels)


In [7]:

# Model with Focal Loss predictions
pred_focal = model_focal.predict(test_X)
y_pred_focal = tf.argmax(pred_focal.logits, axis=1)

1/1 [==============================] - 2s 2s/step


In [9]:

# Focal Loss model metrics
precision_focal = precision_score(test_y, y_pred_focal)
recall_focal = recall_score(test_y, y_pred_focal)
f1_focal = f1_score(test_y, y_pred_focal)
roc_auc_focal = roc_auc_score(test_y, pred_focal.logits[:, 1])

In [10]:
precision_focal

np.float64(0.5)

In [11]:
# Train the default model
model_default.fit(X, y, epochs=3, batch_size=2)



Epoch 1/3


: 

In [ ]:
# Default model predictions
pred_default = model_default.predict(test_X)
y_pred_default = tf.argmax(pred_default.logits, axis=1)


In [ ]:
# Default model metrics
precision_default = precision_score(test_y, y_pred_default)
recall_default = recall_score(test_y, y_pred_default)
f1_default = f1_score(test_y, y_pred_default)
roc_auc_default = roc_auc_score(test_y, pred_default.logits[:, 1])
